In [1]:
import pandas as pd

import shapely.geometry as geometry
import plotly.express as px
import geopandas as gpd
import json

base_path = '../../FINAL data/'
my_data_path = '../../MY data/'

#### final_rosleshoz.csv


In [145]:
file_name = "final_rosleshoz.csv"
path = base_path + "/" + file_name
final_rosselhoz_df = pd.read_csv(path)

In [146]:
final_rosselhoz_df.head()

,new_fire_id,code,coords,locality_distance,lesn,date_start,area_registr,date_end,status,area_all,...,duration,month,status_category,area_union,count_ignitions,points,_typename,geometry,type,ids
0,7463,к-440 (112108),"53° 04' 26"" с.ш.\n102° 32' 53"" в.д.","н.п. Аларь, азимут 38°, удаление 0.5 км",Аларское,2020-04-08,90.0,2020-04-09,ликвидирован 14.04.2020,454,...,1,4,ликвидирован,4.54,1,NaN,Аларь,MULTIPOLYGON (((11416662.69007241 6997742.4409...,НП,[7463]
1,8614,к-1605 (213710),"53° 11' 49"" с.ш.\n102° 14' 53"" в.д.","н.п. Аляты, азимут 140°, удаление 2.4 км",Аларское,2020-06-29,80.0,2020-06-29,ликвидирован 4.07.2020,82,...,0,6,ликвидирован,0.82,1,NaN,Аляты,MULTIPOLYGON (((11382654.2483037 7019963.49327...,НП,[8614]
2,4406,к-600 (169231),"53° 27' 54"" с.ш.\n103° 21' 29"" в.д.","н.п. Ангарский, азимут 32°, удаление 0.6 км",Аларское,2019-04-29,17.0,2019-04-29,ликвидирован 4.05.2019,17,...,0,4,ликвидирован,0.17,1,NaN,Ангарский,MULTIPOLYGON (((11505972.26910861 7069683.9770...,НП,[4406]
3,7567,к-539 (116005),"53° 25' 16"" с.ш.\n102° 57' 43"" в.д.","н.п. Табарсук, азимут 117°, удаление 4.4 км",Аларское,2020-04-09,3.0,2020-04-09,ликвидирован 14.04.2020,28,...,0,4,ликвидирован,0.28,1,NaN,Дута,MULTIPOLYGON (((11461935.80177199 7061539.7212...,НП,[7567]
4,8808,к-1797 (218004),"53° 18' 22"" с.ш.\n103° 00' 29"" в.д.","н.п. Егоровская, азимут 252°, удаление 0.8 км",Аларское,2020-07-06,6.0,2020-07-06,ликвидирован 11.07.2020,7,...,0,7,ликвидирован,0.07,1,NaN,Егоровская,MULTIPOLYGON (((11466936.57961523 7039954.4280...,НП,[8808]


Разница между area и are_union есть


In [147]:
final_rosselhoz_df[final_rosselhoz_df['area_all_km']
                   != final_rosselhoz_df['area_union']][['area_union', 'area_all_km', 'area_all']]

,area_union,area_all_km,area_all
67,0.06,0.03,3
82,0.06,0.03,3
140,487.17,481.15,48115
160,47.20,46.30,4630
201,38.77,38.35,3835
...,...,...,...
9104,0.06,0.03,3
9105,0.06,0.03,3
9106,0.06,0.03,3
9177,0.17,0.14,14


Удалим колонки, дублирующие другие колонки по площади.


In [148]:
final_rosselhoz_df = final_rosselhoz_df.drop(
    columns=["coords", "area_all_km", "area_forest_km", "count_ignitions"])

Ниже видно, что есть id пожара не обязательно равен id нового пожара.


In [149]:
len(final_rosselhoz_df[final_rosselhoz_df['id']
    != final_rosselhoz_df['new_fire_id']])

120

In [150]:
len(final_rosselhoz_df[final_rosselhoz_df['area_all']
    != final_rosselhoz_df['delta_area_all']])

3

Поажры, у которых начало и конец в разных годах. Year — год начала. То же касается месяца.


In [151]:
final_rosselhoz_df[
    (pd.to_datetime(
        final_rosselhoz_df['date_start'], format='%Y-%m-%d').dt.month == 12)
    & (pd.to_datetime(
        final_rosselhoz_df['date_end'], format='%Y-%m-%d').dt.month == 1)][['date_start', 'date_end', 'year']]

,date_start,date_end,year
1296,2018-12-12,2019-01-05,2018
1391,2018-12-24,2019-01-16,2018
1450,2018-12-21,2019-01-09,2018
4539,2018-12-30,2019-01-06,2018
9274,2020-12-04,2021-01-11,2020
9275,2020-12-05,2021-01-11,2020


Удалим дублирование по времени


In [152]:
final_rosselhoz_df = final_rosselhoz_df.drop(
    columns=["day_start", "day_end", "year", "month", "duration"])

In [153]:
final_rosselhoz_df.head()

,new_fire_id,code,locality_distance,lesn,date_start,area_registr,date_end,status,area_all,area_forest,...,id,geometry_point,buffer,status_category,area_union,points,_typename,geometry,type,ids
0,7463,к-440 (112108),"н.п. Аларь, азимут 38°, удаление 0.5 км",Аларское,2020-04-08,90.0,2020-04-09,ликвидирован 14.04.2020,454,0,...,7463,POINT (11415597.32628477 6996677.077121332),"POLYGON ((11417728.05386004 6998807.804696598,...",ликвидирован,4.54,NaN,Аларь,MULTIPOLYGON (((11416662.69007241 6997742.4409...,НП,[7463]
1,8614,к-1605 (213710),"н.п. Аляты, азимут 140°, удаление 2.4 км",Аларское,2020-06-29,80.0,2020-06-29,ликвидирован 4.07.2020,82,0,...,8614,POINT (11382201.47904679 7019510.724017813),"POLYGON ((11383107.01756061 7020416.262531627,...",ликвидирован,0.82,NaN,Аляты,MULTIPOLYGON (((11382654.2483037 7019963.49327...,НП,[8614]
2,4406,к-600 (169231),"н.п. Ангарский, азимут 32°, удаление 0.6 км",Аларское,2019-04-29,17.0,2019-04-29,ликвидирован 4.05.2019,17,1,...,4406,POINT (11505766.11382733 7069477.82176244),"POLYGON ((11506178.42438989 7069890.132325002,...",ликвидирован,0.17,NaN,Ангарский,MULTIPOLYGON (((11505972.26910861 7069683.9770...,НП,[4406]
3,7567,к-539 (116005),"н.п. Табарсук, азимут 117°, удаление 4.4 км",Аларское,2020-04-09,3.0,2020-04-09,ликвидирован 14.04.2020,28,21,...,7567,POINT (11461671.22664088 7061275.146122432),"POLYGON ((11462200.37690309 7061804.296384646,...",ликвидирован,0.28,NaN,Дута,MULTIPOLYGON (((11461935.80177199 7061539.7212...,НП,[7567]
4,8808,к-1797 (218004),"н.п. Егоровская, азимут 252°, удаление 0.8 км",Аларское,2020-07-06,6.0,2020-07-06,ликвидирован 11.07.2020,7,2,...,8808,POINT (11466804.29204968 7039822.14047591),"POLYGON ((11467068.86718079 7040086.715607016,...",ликвидирован,0.07,NaN,Егоровская,MULTIPOLYGON (((11466936.57961523 7039954.4280...,НП,[8808]


Удалим лишние поля


In [154]:
final_rosselhoz_df = final_rosselhoz_df.drop(
    columns=["points", 'ids'])

In [155]:
final_rosselhoz_df.head()

,new_fire_id,code,locality_distance,lesn,date_start,area_registr,date_end,status,area_all,area_forest,...,lon,forest,id,geometry_point,buffer,status_category,area_union,_typename,geometry,type
0,7463,к-440 (112108),"н.п. Аларь, азимут 38°, удаление 0.5 км",Аларское,2020-04-08,90.0,2020-04-09,ликвидирован 14.04.2020,454,0,...,102.548056,0.0,7463,POINT (11415597.32628477 6996677.077121332),"POLYGON ((11417728.05386004 6998807.804696598,...",ликвидирован,4.54,Аларь,MULTIPOLYGON (((11416662.69007241 6997742.4409...,НП
1,8614,к-1605 (213710),"н.п. Аляты, азимут 140°, удаление 2.4 км",Аларское,2020-06-29,80.0,2020-06-29,ликвидирован 4.07.2020,82,0,...,102.248056,0.0,8614,POINT (11382201.47904679 7019510.724017813),"POLYGON ((11383107.01756061 7020416.262531627,...",ликвидирован,0.82,Аляты,MULTIPOLYGON (((11382654.2483037 7019963.49327...,НП
2,4406,к-600 (169231),"н.п. Ангарский, азимут 32°, удаление 0.6 км",Аларское,2019-04-29,17.0,2019-04-29,ликвидирован 4.05.2019,17,1,...,103.358056,1.0,4406,POINT (11505766.11382733 7069477.82176244),"POLYGON ((11506178.42438989 7069890.132325002,...",ликвидирован,0.17,Ангарский,MULTIPOLYGON (((11505972.26910861 7069683.9770...,НП
3,7567,к-539 (116005),"н.п. Табарсук, азимут 117°, удаление 4.4 км",Аларское,2020-04-09,3.0,2020-04-09,ликвидирован 14.04.2020,28,21,...,102.961944,1.0,7567,POINT (11461671.22664088 7061275.146122432),"POLYGON ((11462200.37690309 7061804.296384646,...",ликвидирован,0.28,Дута,MULTIPOLYGON (((11461935.80177199 7061539.7212...,НП
4,8808,к-1797 (218004),"н.п. Егоровская, азимут 252°, удаление 0.8 км",Аларское,2020-07-06,6.0,2020-07-06,ликвидирован 11.07.2020,7,2,...,103.008056,1.0,8808,POINT (11466804.29204968 7039822.14047591),"POLYGON ((11467068.86718079 7040086.715607016,...",ликвидирован,0.07,Егоровская,MULTIPOLYGON (((11466936.57961523 7039954.4280...,НП


Записываем в csv


In [156]:
path = my_data_path + "/" + file_name
final_rosselhoz_df.to_csv(path, index=False)

#### forest_types_processed.csv


#### lesnye_kv_3051.nearest_weather_station-forest_types.csv


In [157]:
file_name = "lesnye_kv_3051.nearest_weather_station-forest_types.csv"
path = base_path + "/Лесные кварталы/" + file_name
nws_df = pd.read_csv(path)

C:\Users\pilno\AppData\Local\Temp\ipykernel_25620\1441391567.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  nws_df = pd.read_csv(path)


In [158]:
nws_df.head()

,name_in,name_uch_l,in_dacha,name_in_ru,dacha_ru,uch_l_ru,kv_num,lesn1,lesn2,lesn3,...,geom,weather_stations,nearest_weather_station,lesn_path,lesn_path_type,distance,forest_zone,forest_seed_zoning_zone,lesn_path_FH,forest_hazard_classes
0,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,28,Аларское,Аларское,NaN,...,0106000020E6100000010000000103000000010000000A...,"30616,30606,30703,30617,30611,30603,30605,3061...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[4.0]
1,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,21,Аларское,Аларское,NaN,...,0106000020E61000000100000001030000000100000009...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
2,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,22,Аларское,Аларское,NaN,...,0106000020E61000000100000001030000000100000008...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
3,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,23,Аларское,Аларское,NaN,...,0106000020E6100000010000000103000000010000000A...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
4,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,24,Аларское,Аларское,NaN,...,0106000020E61000000100000001030000000100000009...,"30606,30616,30703,30617,30611,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]


In [159]:
nws_df[nws_df['kv'] != nws_df['kv_num']]

,name_in,name_uch_l,in_dacha,name_in_ru,dacha_ru,uch_l_ru,kv_num,lesn1,lesn2,lesn3,...,geom,weather_stations,nearest_weather_station,lesn_path,lesn_path_type,distance,forest_zone,forest_seed_zoning_zone,lesn_path_FH,forest_hazard_classes


In [160]:
nws_df = nws_df.drop(columns=["kv_num", 'distance', 'lesn1', 'lesn2', 'lesn3'])

In [161]:
nws_df.head()

,name_in,name_uch_l,in_dacha,name_in_ru,dacha_ru,uch_l_ru,kv,id,geom,weather_stations,nearest_weather_station,lesn_path,lesn_path_type,forest_zone,forest_seed_zoning_zone,lesn_path_FH,forest_hazard_classes
0,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,28,22562,0106000020E6100000010000000103000000010000000A...,"30616,30606,30703,30617,30611,30603,30605,3061...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[4.0]
1,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,21,22552,0106000020E61000000100000001030000000100000009...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
2,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,22,22553,0106000020E61000000100000001030000000100000008...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
3,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,23,22554,0106000020E6100000010000000103000000010000000A...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
4,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,24,22555,0106000020E61000000100000001030000000100000009...,"30606,30616,30703,30617,30611,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]


#### forest_types_processed.csv


In [162]:
file_name = "forest_types_processed.csv"
path = base_path + "/Лесные кварталы/" + file_name
ftp_df = pd.read_csv(path, sep=";")

In [163]:
ftp_df.head()

,name_in,name_in_ru,uch_l_ru,name_in.1,forest_zone,forest_seed_zoning_zone,kv
0,Alarskoe,Аларское,Приморское,Приморская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
1,Alarskoe,Аларское,Аларское,Аларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
2,Alarskoe,Аларское,Алятское,Алятская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
3,Alarskoe,Аларское,Заречное,Заречная,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
4,Alarskoe,Ангарское,Ушаковское,Худяковская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."


In [164]:
for n in ftp_df['name_in.1']:
    t = ftp_df[ftp_df['name_in.1'] == n]['forest_seed_zoning_zone'].unique()
    print(f'{n} {t}')

# ftp_df['forest_seed_zoning_zone']

Приморская ['сосна-18, ель-10, лиственница-10, кедр-6'
 'сосна-18, ель-10, лиственница-10, кедр-9']
Аларская ['сосна-18, ель-10, лиственница-10, кедр-6']
Алятская ['сосна-18, ель-10, лиственница-10, кедр-6']
Заречная ['сосна-18, ель-10, лиственница-10, кедр-6']
Худяковская ['сосна-18, ель-10, лиственница-10, кедр-9']
Добролетская ['сосна-18, ель-10, лиственница-10, кедр-9']
Ключевская ['сосна-18, ель-10, лиственница-10, кедр-9']
Ангарская ['сосна-18, ель-10, лиственница-10, кедр-9']
Тальцинская ['сосна-18, ель-10, лиственница-10, кедр-9']
Технический участок  №10 ГУСХП ОПХ Иркутское ['сосна-18, ель-10, лиственница-10, кедр-9']
Балаганская ['сосна-18, ель-9, лиственница-10, кедр-9']
Технический участок №1 Ленинский путь ['сосна-18, ель-9, лиственница-10, кедр-9']
Технический участок №3 Балаганский ['сосна-18, ель-9, лиственница-10, кедр-9']
Балаганская ['сосна-18, ель-9, лиственница-10, кедр-9']
Кадинская ['сосна-18, ель-9, лиственница-10, кедр-9']
Карахунская ['сосна-18, ель-9, листвен

In [165]:
ftp_df.rename(columns={'name_in.1': 'dacha'})

,name_in,name_in_ru,uch_l_ru,dacha,forest_zone,forest_seed_zoning_zone,kv
0,Alarskoe,Аларское,Приморское,Приморская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
1,Alarskoe,Аларское,Аларское,Аларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
2,Alarskoe,Аларское,Алятское,Алятская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
3,Alarskoe,Аларское,Заречное,Заречная,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
4,Alarskoe,Ангарское,Ушаковское,Худяковская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
...,...,...,...,...,...,...,...
639,Shelehovskoe,Шелеховское,Шелеховское,Олхинская,Южно-Сибирская горная,"сосна-18, ель-12, лиственница-10, кедр-9","65,66,67,68,69,70,71,72,73,74,77,78,79,80,81,8..."
640,Shelehovskoe,Шелеховское,Шелеховское,Подкаменская,Южно-Сибирская горная,"сосна-18, ель-12, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
641,Shelehovskoe,Шелеховское,Шелеховское,Мотская,Южно-Сибирская горная,"сосна-18, ель-12, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
642,Shelehovskoe,Шелеховское,Шелеховское,Олхинская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."


In [166]:
# ftp_df.drop()

In [167]:
ftp_df.head()

,name_in,name_in_ru,uch_l_ru,name_in.1,forest_zone,forest_seed_zoning_zone,kv
0,Alarskoe,Аларское,Приморское,Приморская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
1,Alarskoe,Аларское,Аларское,Аларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
2,Alarskoe,Аларское,Алятское,Алятская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
3,Alarskoe,Аларское,Заречное,Заречная,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
4,Alarskoe,Ангарское,Ушаковское,Худяковская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."


#### localities_irk_obl.geojson

Geopandas не воспринимает массивы полигонов, необходимо переписать файл

In [168]:
file_name = 'localities_irk_obl.geojson'
path = base_path + "/" + file_name

loc_df = gpd.read_file(path, ignore_geometry=True)

In [169]:
loc_df.head()

,name,type,name_MO,code,distance,ado,id,query,address,geometry,poly,valid,locality
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,Аларь село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0103000020E6100000010000002E000000726891ED7CA1...,1.0,1
1,Александровск,село сельский,Муниципальное образование «Александровск»,25123904001,10,Аларский,01.���,Александровск село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0103000020E61000000100000050000000C30DF8FC30A9...,1.0,1
2,Алзобей,деревня сельский,Муниципальное образование «Аларь»,25123902002,36,Аларский,01.���,Алзобей деревня Аларский иркутская область,"Россия, Иркутская область, Аларский район, дер...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0103000020E6100000010000001A00000056F0DB10E3A7...,1.0,1
3,Аляты,село сельский,Муниципальное образование «Аляты»,25123907001,55,Аларский,01.���,Аляты село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0106000020E61000000200000001030000000100000017...,1.0,1
4,Ангарский,поселок сельский,Муниципальное образование «Ангарский»,25123910001,55,Аларский,01.���,Ангарский поселок Аларский иркутская область,"Россия, Иркутская область, Аларский район, пос...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0103000020E6100000010000001500000074ECA012D7D5...,1.0,1


In [170]:
def str_to_multipoly(lst):
    # В некоторых строках вместо ',' стоят ';', тогда json не читается
    lst = lst.replace(';', ',')
    j = json.loads(lst)
    polygons = []
    for p in j:
        p = geometry.shape(p)
        polygons.append(p)

    polygons = geometry.MultiPolygon(polygons)

    return geometry.MultiPolygon(polygons)

In [171]:
geom = []

for index, rows in loc_df.iterrows():
    mlp = str_to_multipoly(loc_df.loc[index].geometry)
    # loc_df.at[index, 'geometry'] = mlp
    geom.append(mlp)

In [172]:
gdf = gpd.GeoDataFrame(
    loc_df, geometry=geom, crs="EPSG:4326"
)

In [173]:
gdf = gdf.drop(gdf[gdf.geometry.is_empty].index)

In [174]:
gdf.head()

,name,type,name_MO,code,distance,ado,id,query,address,geometry,poly,valid,locality
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,Аларь село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","MULTIPOLYGON (((102.52325 53.07457, 102.52335 ...",0103000020E6100000010000002E000000726891ED7CA1...,1.0,1
1,Александровск,село сельский,Муниципальное образование «Александровск»,25123904001,10,Аларский,01.���,Александровск село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","MULTIPOLYGON (((102.64361 53.33077, 102.64394 ...",0103000020E61000000100000050000000C30DF8FC30A9...,1.0,1
2,Алзобей,деревня сельский,Муниципальное образование «Аларь»,25123902002,36,Аларский,01.���,Алзобей деревня Аларский иркутская область,"Россия, Иркутская область, Аларский район, дер...","MULTIPOLYGON (((102.62323 53.11388, 102.62461 ...",0103000020E6100000010000001A00000056F0DB10E3A7...,1.0,1
3,Аляты,село сельский,Муниципальное образование «Аляты»,25123907001,55,Аларский,01.���,Аляты село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","MULTIPOLYGON (((102.22077 53.19755, 102.22384 ...",0106000020E61000000200000001030000000100000017...,1.0,1
4,Ангарский,поселок сельский,Муниципальное образование «Ангарский»,25123910001,55,Аларский,01.���,Ангарский поселок Аларский иркутская область,"Россия, Иркутская область, Аларский район, пос...","MULTIPOLYGON (((103.34125 53.46178, 103.34265 ...",0103000020E6100000010000001500000074ECA012D7D5...,1.0,1


In [175]:
gdf.drop(columns=['query', 'locality'])

,name,type,name_MO,code,distance,ado,id,address,geometry,poly,valid
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"Россия, Иркутская область, Аларский район, сел...","MULTIPOLYGON (((102.52325 53.07457, 102.52335 ...",0103000020E6100000010000002E000000726891ED7CA1...,1.0
1,Александровск,село сельский,Муниципальное образование «Александровск»,25123904001,10,Аларский,01.���,"Россия, Иркутская область, Аларский район, сел...","MULTIPOLYGON (((102.64361 53.33077, 102.64394 ...",0103000020E61000000100000050000000C30DF8FC30A9...,1.0
2,Алзобей,деревня сельский,Муниципальное образование «Аларь»,25123902002,36,Аларский,01.���,"Россия, Иркутская область, Аларский район, дер...","MULTIPOLYGON (((102.62323 53.11388, 102.62461 ...",0103000020E6100000010000001A00000056F0DB10E3A7...,1.0
3,Аляты,село сельский,Муниципальное образование «Аляты»,25123907001,55,Аларский,01.���,"Россия, Иркутская область, Аларский район, сел...","MULTIPOLYGON (((102.22077 53.19755, 102.22384 ...",0106000020E61000000200000001030000000100000017...,1.0
4,Ангарский,поселок сельский,Муниципальное образование «Ангарский»,25123910001,55,Аларский,01.���,"Россия, Иркутская область, Аларский район, пос...","MULTIPOLYGON (((103.34125 53.46178, 103.34265 ...",0103000020E6100000010000001500000074ECA012D7D5...,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1537,Тулун,город,Муниципальное образование – «город Тулун»,25432000000,428,Тулун,39.0,"Россия, Иркутская область, Тулун","MULTIPOLYGON (((100.72851 54.49430, 100.72937 ...",0106000020E61000000200000001030000000100000029...,1.0
1538,Усолье-Сибирское,город,Муниципальное образование «город Усолье-Сибирс...,25436000000,84,Усолье-Сибирское,40.0,"Россия, Иркутская область, Усолье-Сибирское","MULTIPOLYGON (((103.65076 52.83827, 103.65130 ...",0106000020E6100000030000000103000000010000000C...,1.0
1539,Усть-Илимск,город,Муниципальное образование город Усть-Илимск,25438000000,893,Усть-Илимск,41.0,"Россия, Иркутская область, Усть-Илимск","MULTIPOLYGON (((102.84220 58.05423, 102.84229 ...",0106000020E61000000200000001030000000100000072...,1.0
1540,Черемхово,город,Муниципальное образование «город Черемхово»,25445000000,157,Черемхово,42.0,"Россия, Иркутская область, Черемхово","MULTIPOLYGON (((103.20563 53.13499, 103.20555 ...",0106000020E61000000A0000000103000000010000000A...,1.0


In [176]:
gdf[gdf.geometry == None]

,name,type,name_MO,code,distance,ado,id,query,address,geometry,poly,valid,locality


In [177]:
path = my_data_path + "/" + file_name
gdf.to_file(path, driver='GeoJSON', index=False)

#### user_schema.lesnye_kv_3051.geojson

In [178]:
file_name = "user_schema.lesnye_kv_3051.geojson"

path_to_json = base_path + "Лесные кварталы/" + file_name

gdf = gpd.read_file(path_to_json)

In [179]:
gdf.head()

,name_in,name_uch_l,in_dacha,name_in_ru,dacha_ru,uch_l_ru,kv_num,commentary,lesn1,lesn2,...,id,is_deleted,classname,created_by,edited_by,edited_on,created_on,published,geom:1,geometry
0,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,28,None,Аларское,Аларское,...,22562,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E6100000010000000103000000010000000A...,"MULTIPOLYGON (((102.37619 53.25439, 102.37240 ..."
1,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,21,None,Аларское,Аларское,...,22552,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E61000000100000001030000000100000009...,"MULTIPOLYGON (((102.22014 53.31237, 102.22171 ..."
2,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,22,None,Аларское,Аларское,...,22553,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E61000000100000001030000000100000008...,"MULTIPOLYGON (((102.23036 53.29569, 102.21632 ..."
3,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,23,None,Аларское,Аларское,...,22554,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E6100000010000000103000000010000000A...,"MULTIPOLYGON (((102.25694 53.29010, 102.25134 ..."
4,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,24,None,Аларское,Аларское,...,22555,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E61000000100000001030000000100000009...,"MULTIPOLYGON (((102.24487 53.28587, 102.24644 ..."


In [180]:
gdf['id'].unique()

array([22562, 22552, 22553, ..., 17483, 17485, 17486], dtype=int64)

In [181]:
gdf.columns

Index(['name_in', 'name_uch_l', 'in_dacha', 'name_in_ru', 'dacha_ru',
       'uch_l_ru', 'kv_num', 'commentary', 'lesn1', 'lesn2', 'lesn3', 'kv',
       'id', 'is_deleted', 'classname', 'created_by', 'edited_by', 'edited_on',
       'created_on', 'published', 'geom:1', 'geometry'],
      dtype='object')

In [182]:
gdf = gdf.drop(columns=['commentary', 'lesn1', 'lesn2', 'lesn3',
         'geom:1', 'published', 'created_on', 'edited_on', 'created_by', 'edited_by', 'classname', 'is_deleted', 'kv'])

In [183]:
gdf = gdf.drop(gdf[gdf['geometry'].is_empty].index)

In [184]:
gdf.head()

,name_in,name_uch_l,in_dacha,name_in_ru,dacha_ru,uch_l_ru,kv_num,id,geometry
0,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,28,22562,"MULTIPOLYGON (((102.37619 53.25439, 102.37240 ..."
1,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,21,22552,"MULTIPOLYGON (((102.22014 53.31237, 102.22171 ..."
2,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,22,22553,"MULTIPOLYGON (((102.23036 53.29569, 102.21632 ..."
3,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,23,22554,"MULTIPOLYGON (((102.25694 53.29010, 102.25134 ..."
4,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,24,22555,"MULTIPOLYGON (((102.24487 53.28587, 102.24644 ..."


In [185]:
path = my_data_path + "Лесные кварталы/" + file_name
gdf.to_file(path, driver='GeoJSON', index=False)

#### zhd_roads.geojson


In [2]:
file_name = "zhd_roads.geojson"

path_to_json = base_path + "/metadata/" + file_name

df_rail = gpd.read_file(path_to_json)
df_rail.head()


,id,is_deleted,symbol,created_by,edited_by,edited_on,created_on,published,geom:1,geometry
0,1,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.062,2017-04-18 18:20:56.062000+09:00,f,0105000020E61000000100000001020000001300000004...,"LINESTRING (117.01367 52.06945, 117.07109 52.0..."
1,2,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.133,2017-04-18 18:20:56.133000+09:00,f,0105000020E61000000100000001020000002300000058...,"LINESTRING (91.10172 53.57645, 90.95557 53.575..."
2,3,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.135,2017-04-18 18:20:56.135000+09:00,f,0105000020E61000000100000001020000005F000000DA...,"LINESTRING (97.95192 55.94242, 97.84468 55.934..."
3,4,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.136,2017-04-18 18:20:56.136000+09:00,f,0105000020E6100000010000000102000000050000004E...,"LINESTRING (118.12918 50.25033, 118.10700 50.2..."
4,5,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.138,2017-04-18 18:20:56.138000+09:00,f,0105000020E6100000010000000102000000800000003D...,"LINESTRING (114.52750 51.64029, 114.52327 51.6..."


In [3]:
df_rail[df_rail['is_deleted']!='f']

,id,is_deleted,symbol,created_by,edited_by,edited_on,created_on,published,geom:1,geometry


In [4]:
df_rail[df_rail['symbol']!=None]

,id,is_deleted,symbol,created_by,edited_by,edited_on,created_on,published,geom:1,geometry
0,1,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.062,2017-04-18 18:20:56.062000+09:00,f,0105000020E61000000100000001020000001300000004...,"LINESTRING (117.01367 52.06945, 117.07109 52.0..."
1,2,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.133,2017-04-18 18:20:56.133000+09:00,f,0105000020E61000000100000001020000002300000058...,"LINESTRING (91.10172 53.57645, 90.95557 53.575..."
2,3,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.135,2017-04-18 18:20:56.135000+09:00,f,0105000020E61000000100000001020000005F000000DA...,"LINESTRING (97.95192 55.94242, 97.84468 55.934..."
3,4,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.136,2017-04-18 18:20:56.136000+09:00,f,0105000020E6100000010000000102000000050000004E...,"LINESTRING (118.12918 50.25033, 118.10700 50.2..."
4,5,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.138,2017-04-18 18:20:56.138000+09:00,f,0105000020E6100000010000000102000000800000003D...,"LINESTRING (114.52750 51.64029, 114.52327 51.6..."
5,6,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.140,2017-04-18 18:20:56.140000+09:00,f,0105000020E61000000100000001020000006400000098...,"LINESTRING (107.63455 51.81058, 107.58977 51.7..."
6,7,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.142,2017-04-18 18:20:56.142000+09:00,f,0105000020E61000000100000001020000004B00000014...,"LINESTRING (105.88475 49.20835, 105.84591 49.1..."
7,8,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.143,2017-04-18 18:20:56.143000+09:00,f,0105000020E6100000010000000102000000840000003C...,"LINESTRING (94.59940 55.91340, 94.55504 55.903..."
8,9,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.145,2017-04-18 18:20:56.145000+09:00,f,0105000020E6100000010000000102000000A3030000F2...,"LINESTRING (98.05490 55.91693, 98.26260 56.056..."
9,10,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.150,2017-04-18 18:20:56.150000+09:00,f,0105000020E6100000010000000102000000080000006C...,"LINESTRING (104.42041 49.03006, 104.40679 49.0..."


In [5]:
df_rail = df_rail.drop(columns=['is_deleted', 'published', 'edited_by',
                       'edited_on', 'created_by', 'created_on', 'published', 'symbol'])

In [6]:
df_rail.head()

,id,geom:1,geometry
0,1,0105000020E61000000100000001020000001300000004...,"LINESTRING (117.01367 52.06945, 117.07109 52.0..."
1,2,0105000020E61000000100000001020000002300000058...,"LINESTRING (91.10172 53.57645, 90.95557 53.575..."
2,3,0105000020E61000000100000001020000005F000000DA...,"LINESTRING (97.95192 55.94242, 97.84468 55.934..."
3,4,0105000020E6100000010000000102000000050000004E...,"LINESTRING (118.12918 50.25033, 118.10700 50.2..."
4,5,0105000020E6100000010000000102000000800000003D...,"LINESTRING (114.52750 51.64029, 114.52327 51.6..."


In [7]:
df_rail[df_rail['geometry'].is_empty]

,id,geom:1,geometry


In [8]:
path = my_data_path + "metadata/" + file_name
df_rail.to_file(path, driver='GeoJSON', index=False)